In [41]:
import numpy as np
import pandas as pd
import keras
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from arch import arch_model
from sklearn.preprocessing import MinMaxScaler
from keras import models
from keras import layers
from sklearn.metrics import mean_squared_error
import random


In [42]:

def find_best_arima_order(time_series, p_range=(0, 3), d_range=(0, 1), q_range=(0, 3)):
    """
    Find the best ARIMA (p, d, q) order based on the lowest AIC score.
    
    Parameters:
    time_series (pd.Series): The time series data (stock prices or log returns).
    p_range (tuple): Range of p values to try.
    d_range (tuple): Range of d values to try.
    q_range (tuple): Range of q values to try.
    
    Returns:
    tuple: Best (p, d, q) values based on AIC.
    """
    best_aic = float('inf')
    best_order = None

    for p in range(p_range[0], p_range[1] + 1):
        for d in range(d_range[0], d_range[1] + 1):
            for q in range(q_range[0], q_range[1] + 1):
                try:
                    # Fit ARIMA model
                    model = ARIMA(time_series, order=(p, d, q))
                    fitted_model = model.fit()

                    # Check AIC and store the best order
                    if fitted_model.aic < best_aic:
                        best_aic = fitted_model.aic
                        best_order = (p, d, q)
                except:
                    continue

    return best_order


In [43]:
def decompose_data(data):
    """
    Decompose time series data into linear, seasonal (stable) parts and residual (unstable) parts.
    Uses ARIMA for long-term trends and GARCH for modeling volatility clusters.
    
    Parameters:
    - data (pd.Series): Stock price time series.
    
    Returns:
    - long_term: ARIMA modeled long-term trend data.
    - seasonal: GARCH modeled volatility seasonal data.
    - residual: Unstable (residual) part of the time series.
    """
    # Find the best ARIMA order dynamically
    best_arima_order = find_best_arima_order(data, p_range=(0, 3), d_range=(0, 1), q_range=(0, 3))
    print(f"Best ARIMA order found: {best_arima_order}")

    # Fit ARIMA model (long-term trend) using the best ARIMA order
    arima_model = ARIMA(data, order=best_arima_order)
    arima_fitted = arima_model.fit()
    long_term = arima_fitted.fittedvalues

    print(arima_fitted.summary())

    # Fit GARCH model (seasonal effects)
    garch_model = arch_model(data - long_term, vol='Garch', p=1, q=1)
    # Fit the model with optimization options
    options = {'maxiter': 1000, 'disp': False}  # Control the optimization process
    garch_fitted = garch_model.fit(update_freq=5, disp='off', options=options)
    seasonal = garch_fitted.conditional_volatility

    if garch_fitted.convergence_flag == 0:
        print("GARCH Model converged successfully")
    else:
        print("Warning: GARCH Model did not converge")

    print(garch_fitted.summary())

    # Calculate residual
    residual = data - long_term - seasonal

    return long_term, seasonal, residual


In [44]:
def split_data(long_term, seasonal, residual, train_size=0.75):
    """
    Split data into train and test sets for each part of the time series.
    
    Parameters:
    - long_term, seasonal, residual: Components of the decomposed time series.
    - train_size (float): Proportion of data to use for training.
    
    Returns:
    - Train and test sets for each part.
    """
    split_index = int(len(long_term) * train_size)

    train_lt, test_lt = long_term[:split_index], long_term[split_index:]
    train_st, test_st = seasonal[:split_index], seasonal[split_index:]
    train_res, test_res = residual[:split_index], residual[split_index:]

    return (train_lt, test_lt), (train_st, test_st), (train_res, test_res)


In [45]:
def prepare_lstm_data(data, look_back=5):
    """
    Prepare data for LSTM training.
    
    Parameters:
    - data (pd.DataFrame): Input data for LSTM.
    - look_back (int): Number of time steps to consider for the LSTM input.
    
    Returns:
    - X, y: Input and output sequences for LSTM.
    """
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled_data = scaler.fit_transform(data.values.reshape(-1, 1))

    X, y = [], []
    for i in range(len(scaled_data) - look_back):
        X.append(scaled_data[i:i + look_back])
        y.append(scaled_data[i + look_back])

    return np.array(X), np.array(y), scaler


In [46]:
def create_lstm_model(input_shape):
    """
    Define the LSTM model for forecasting the residual data.
    
    Parameters:
    - input_shape (tuple): Shape of the input data (timesteps, features).
    
    Returns:
    - LSTM model.
    """
    model = models.Sequential()
    model.add(layers.LSTM(units=50, input_shape=input_shape))
    model.add(layers.Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model


In [47]:
def sine_cosine_algorithm(pop_size, dimensions, lb, ub, max_iters, fitness_func):
    """
    Sine-Cosine Algorithm to optimize the hyperparameters of LSTM and ARIMA.
    
    Parameters:
    - pop_size: Population size.
    - dimensions: Number of hyperparameters to optimize.
    - lb: Lower bound for each dimension.
    - ub: Upper bound for each dimension.
    - max_iters: Maximum iterations.
    - fitness_func: Fitness function to evaluate each solution.
    
    Returns:
    - Best solution (optimized hyperparameters).
    """
    # Initialize population
    pop = np.random.uniform(lb, ub, (pop_size, dimensions))
    best_solution = None
    best_fitness = float('inf')

    for iteration in range(max_iters):
        for i in range(pop_size):
            # Evaluate fitness of each candidate solution
            fitness = fitness_func(pop[i])

            if fitness < best_fitness:
                best_fitness = fitness
                best_solution = pop[i]

            # Update position based on Sine-Cosine update rule
            r1, r2, r3, r4 = random.random(), random.random(), random.random(), random.random()
            if r4 < 0.5:
                pop[i] += r1 * np.sin(r2) * abs(r3 * best_solution - pop[i])
            else:
                pop[i] += r1 * np.cos(r2) * abs(r3 * best_solution - pop[i])

        print(f"Iteration {iteration}: Best Fitness = {best_fitness}")

    return best_solution


In [48]:
def forecast_arima_garch_lstm_sca(data, best_params, train_data, test_data):
    """
    Forecast stock prices using ARIMA-GARCH-LSTM-SCA model.
    
    Parameters:
    - data: Original stock price data.
    - best_params: Optimized parameters from SCA.
    - train_data: Training data.
    - test_data: Testing data.
    
    Returns:
    - Predictions.
    """
    # Fit ARIMA-GARCH
    p, q = int(best_params[0]), int(best_params[1])
    arima_garch_model = arch_model(train_data, vol='Garch', p=p, q=q)
    fitted_model = arima_garch_model.fit(disp='off')

    # Forecast test data
    forecast_vol = fitted_model.forecast(horizon=len(test_data))

    # Combine results and return
    predictions = test_data + forecast_vol.variance
    return predictions


In [49]:
# Define the fitness function for SCA (minimizing RMSE between actual and predicted values)
def fitness_function(params):
    p, q = int(params[0]), int(params[1])
    arima_garch_model = arch_model(train_lt, vol='Garch', p=p, q=q)
    fitted_model = arima_garch_model.fit(disp='off')

    # Forecast and calculate RMSE
    forecast_vol = fitted_model.forecast(horizon=len(test_lt))
    predictions = test_lt + forecast_vol.variance

    return mean_squared_error(test_lt, predictions)

In [50]:
# Convert log returns back to prices
def log_returns_to_prices(log_returns, initial_price):
    """
    Convert log returns to prices.
    
    Parameters:
    log_returns (np.array): Array of log returns.
    initial_price (float): The initial price from which to start the conversion.
    
    Returns:
    np.array: Array of predicted prices.
    """
    predicted_prices = np.zeros_like(log_returns)
    predicted_prices[0] = initial_price
    for t in range(1, len(log_returns)):
        predicted_prices[t] = predicted_prices[t-1] * np.exp(log_returns[t])
    return predicted_prices

In [51]:
def combine_predictions(arima_garch_predictions, lstm_predictions):
    """
    Combine ARIMA-GARCH and LSTM predictions using a direct summation method.
    
    Parameters:
    - arima_garch_predictions (np.array or pd.Series): Predictions from the ARIMA-GARCH model.
    - lstm_predictions (np.array or pd.Series): Predictions from the LSTM model.
    
    Returns:
    - combined_predictions (np.array): Summation of both predictions.
    """
    # Ensure both prediction arrays are the same shape
    if len(arima_garch_predictions) != len(lstm_predictions):
        raise ValueError("The predictions from ARIMA-GARCH and LSTM models must have the same length.")

    # Combine the predictions by adding them together
    combined_predictions = arima_garch_predictions + lstm_predictions

    return combined_predictions


In [53]:
import warnings
from arch.utility.exceptions import DataScaleWarning

# Suppress the DataScaleWarning
warnings.filterwarnings("ignore", category=DataScaleWarning)

# Load preprocessed stock data with raw prices (not log returns)
data = pd.read_csv('../data/processed/AAPL_processed.csv')
prices = data['Adj Close']

# Decompose the price series to extract long-term, seasonal, and residual components
long_term, seasonal, residual = decompose_data(prices)
(train_lt, test_lt), (train_st, test_st), (train_res, test_res) = split_data(long_term, seasonal, residual)

# Define bounds for p and q in ARIMA-GARCH model optimization
lb = [1, 1]  # Lower bounds for ARIMA-GARCH p, q parameters
ub = [5, 5]  # Upper bounds for ARIMA-GARCH p, q parameters

# Optimize ARIMA-GARCH parameters using SCA (Sine Cosine Algorithm)
best_params = sine_cosine_algorithm(pop_size=30, dimensions=2, lb=lb, ub=ub, max_iters=50, fitness_func=fitness_function)

print(f"Best parameters found by SCA: {best_params}")

# Fit ARIMA-GARCH Model using the optimized parameters
# Use the long-term component of the price data (train_lt) in this case
arima_garch_predictions = forecast_arima_garch_lstm_sca(prices, best_params, train_lt, test_lt)

# Prepare LSTM training data (using the residual part from decomposition)
X_train, y_train, scaler = prepare_lstm_data(pd.DataFrame(train_res))

# Create LSTM model based on the input shape of the training data
lstm_model = create_lstm_model(input_shape=(X_train.shape[1], X_train.shape[2]))

# Train the LSTM model on residuals
lstm_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=2)

# Make LSTM predictions
lstm_predictions = lstm_model.predict(X_train)

# Combine ARIMA-GARCH predictions and LSTM residual predictions
# Depending on your strategy (e.g., weighted combination), combine the forecasts
combined_forecast = combine_predictions(arima_garch_predictions, lstm_predictions)
# Ensure that predictions and actual prices have consistent lengths



C:\CoDE\workspace-idea\everything\venv\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\CoDE\workspace-idea\everything\venv\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\CoDE\workspace-idea\everything\venv\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\CoDE\workspace-idea\everything\venv\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.

Best ARIMA order found: (3, 0, 2)


C:\CoDE\workspace-idea\everything\venv\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


                               SARIMAX Results                                
Dep. Variable:              Adj Close   No. Observations:                 3521
Model:                 ARIMA(3, 0, 2)   Log Likelihood               -6264.802
Date:                Sun, 08 Sep 2024   AIC                          12543.605
Time:                        12:19:14   BIC                          12586.770
Sample:                             0   HQIC                         12559.005
                               - 3521                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         57.7277   1381.103      0.042      0.967   -2649.184    2764.640
ar.L1         -0.8595      0.005   -172.215      0.000      -0.869      -0.850
ar.L2          0.8842      0.002    522.320      0.0

ValueError: Found input variables with inconsistent numbers of samples: [881, 1]